# Transfer Learning with TensorFlow: Fine Tuning

In **fine-tuning transfer learning** the pre-trained model weights from another model are unfrozen and tweaked during to better suit your own data.

For feature extraction transfer learning, you may only train the top 1-3 layers of a pre-trained model with your own data, in fine-tuning transfer learning, you might train 1-3+ layers of a pre-trained model (where the '+' indicates that many or all of the layers could be trained).

*Feature extraction transfer learning vs. fine-tuning transfer learning. The main difference between the two is that in fine-tuning, more layers of the pre-trained model get unfrozen and tuned on custom data. This fine-tuning usually takes more data than feature extraction to be effective.*

## What we're going to cover

We're going to go through the follow with TensorFlow:

- Introduce fine-tuning, a type of transfer learning to modify a pre-trained model to be more suited to your data
- Using the Keras Functional API (a differnt way to build models in Keras)
- Using a smaller dataset to experiment faster (e.g. 1-10% of training samples of 10 classes of food)
- Data augmentation (how to make your training dataset more diverse without adding more data)
- Running a series of modelling experiments on our Food Vision data
  - Model 0: a transfer learning model using the Keras Functional API
  - Model 1: a feature extraction transfer learning model on 1% of the data with data augmentation
  - Model 2: a feature extraction transfer learning model on 10% of the data with data augmentation
  - Model 3: a fine-tuned transfer learning model on 10% of the data
  - Model 4: a fine-tuned transfer learning model on 100% of the data
- Introduce the ModelCheckpoint callback to save intermediate training results
- Compare model experiments results using TensorBoard

## How you can use this notebook

You can read through the descriptions and the code (it should all run, except for the cells which error on purpose), but there's a better option.

Write all of the code yourself.

Yes. I'm serious. Create a new notebook, and rewrite each line by yourself. Investigate it, see if you can break it, why does it break?

You don't have to write the text descriptions but writing the code yourself is a great way to get hands-on experience.

Don't worry if you make mistakes, we all do. The way to get better and make less mistakes is to **write more code**.

## Importing Libraries

In [4]:
from src.data import load_and_unzip_data
from src.features import walk_through_dir
import tensorflow as tf

In [2]:
src_link = 'https://storage.googleapis.com/ztm_tf_course/food_vision/10_food_classes_10_percent.zip'
raw_path = 'data/05_tl_with_ft/raw/'
clean_path = 'data/05_tl_with_ft/clean/'

load_and_unzip_data(
    src_link=src_link,
    raw_path=raw_path,
    clean_path=clean_path
)

100%|██████████| 169M/169M [02:29<00:00, 1.13MiB/s] 


Check how many sub-directories and images in out dataset.

In [5]:
path = 'data/05_tl_with_ft/clean/10_food_classes_10_percent'
walk_through_dir(path)

There are 2 directories and 0 images in '10_food_classes_10_percent'.
There are 10 directories and 0 images in 'test'.
There are 0 directories and 250 images in 'ice_cream'.
There are 0 directories and 250 images in 'chicken_curry'.
There are 0 directories and 250 images in 'steak'.
There are 0 directories and 250 images in 'sushi'.
There are 0 directories and 250 images in 'chicken_wings'.
There are 0 directories and 250 images in 'grilled_salmon'.
There are 0 directories and 250 images in 'hamburger'.
There are 0 directories and 250 images in 'pizza'.
There are 0 directories and 250 images in 'ramen'.
There are 0 directories and 250 images in 'fried_rice'.
There are 10 directories and 0 images in 'train'.
There are 0 directories and 75 images in 'ice_cream'.
There are 0 directories and 75 images in 'chicken_curry'.
There are 0 directories and 75 images in 'steak'.
There are 0 directories and 75 images in 'sushi'.
There are 0 directories and 75 images in 'chicken_wings'.
There are 0 d

Creating test and train directory paths.

In [6]:
train_dir = '../data/05_tl_with_ft/clean/10_food_classes_10_percent/train'
test_dir = '../data/05_tl_with_ft/clean/10_food_classes_10_percent/test'

In [7]:
from tensorflow.keras.preprocessing import image_dataset_from_directory

In [8]:
IMG_SIZE =(224, 224)
BATCH_SIZE = 32

train_dataset_10_percent = image_dataset_from_directory(
    directory=train_dir,
    image_size=IMG_SIZE,
    label_mode='categorical',
    batch_size=BATCH_SIZE)

test_dataset = image_dataset_from_directory(
    directory=test_dir,
    image_size=IMG_SIZE,
    label_mode='categorical',
    batch_size=BATCH_SIZE)

Found 750 files belonging to 10 classes.


2021-12-28 13:35:17.633974: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Found 2500 files belonging to 10 classes.


In [9]:
train_dataset_10_percent

<BatchDataset shapes: ((None, 224, 224, 3), (None, 10)), types: (tf.float32, tf.float32)>

Check out the class names of our dataset.

In [10]:
train_dataset_10_percent.class_names

['chicken_curry',
 'chicken_wings',
 'fried_rice',
 'grilled_salmon',
 'hamburger',
 'ice_cream',
 'pizza',
 'ramen',
 'steak',
 'sushi']

In [11]:
for images, labels in train_dataset_10_percent.take(1):
    print(images, labels)

tf.Tensor(
[[[[ 72.20918     54.209183    30.464287  ]
   [ 74.20409     57.178574    31.229595  ]
   [ 97.55613     77.34184     50.06633   ]
   ...
   [ 23.209105    15.780577     5.9948406 ]
   [ 23.734684    16.734684     6.7346845 ]
   [ 18.872366    11.872367     1.8723668 ]]

  [[ 58.096935    41.69388     19.95408   ]
   [104.85204     88.84694     64.13776   ]
   [ 99.39286     81.75        54.107143  ]
   ...
   [ 26.943876    20.943876     8.943876  ]
   [ 26.709156    19.709156     9.709157  ]
   [ 18.49986     11.499861     1.882551  ]]

  [[ 86.59694     70.382645    53.678566  ]
   [ 80.52041     65.234695    45.59184   ]
   [104.35205     87.66327     63.897964  ]
   ...
   [ 27.168327    21.168327     8.831628  ]
   [ 24.056093    17.841808     6.2703795 ]
   [ 19.703997    13.489711     2.0713587 ]]

  ...

  [[ 79.29045     62.948643    36.657867  ]
   [ 87.08149     67.7958      35.556034  ]
   [123.53563     98.36728     58.489754  ]
   ...
   [ 66.75498     43.780